# Domain Identification - System B

The Jupyter notebook presents the workflow for $L_o$ identification presented in the book chapter "Characterization of Domain Formation in Complex Membranes" by Marius F.W. Trollmann and Rainer A. Böckmann.

<div class="alert alert-danger">

<b>Disclaimer!</b>

The shown code examples are written sloley for this project and are adapted to our test systems. There is no guarantee that the notebook or any of the defined functions can be transferred directly to other systems.
    
</div>

In [ ]:
#Import libraries

#Scientific computing
#Tested with NumPy version 1.26.0
#Tested with SciPy version 1.11.3
import numpy as np 
from scipy import stats 
from scipy.spatial import Voronoi, voronoi_plot_2d, ConvexHull

#Unsupervised Machine Learning Algorithms
#Tested with sklearn version 1.3.2
#Tested with hmmlearn version 0.3.0
from sklearn import mixture
from hmmlearn.hmm import GaussianHMM

#Handling of molecular dynamics output
#Tested with MDAnalysis version 2.7.0
#Tested with MemSurfer version 1.1
import MDAnalysis as mda
from MDAnalysis.analysis import distances

#Visualization
#Tested with matplotlib version 3.8.2
import matplotlib.pyplot as plt

#Others
#Tested with tqdm version 4.66.1
from tqdm import tqdm
import sys
import os
import logging 
import pickle

## Load Trajectory

In [ ]:
#Define paths to the trajectory and the topology file
path2xtc="data/md_center_mol_last1mus.xtc" #Here only the last microsecond is used to enable a storage of the trajectory on github
path2tpr="data/mem.tpr"

In [ ]:
uni = mda.Universe(path2tpr, path2xtc)

## Definition of Functions

The following coding cell contains definition of functions used to calculate lipid properties. They are called while looping over the trajectory.

In [ ]:

def order_parameter(chain):
    
    """
    Calculate average Scc order parameters per acyl chain according to the equation:

    S_cc = (3 * cos( theta )^2 - 1) / 2, 

    where theta describes the angle between the z-axis of the system and the vector between two subsequent tail beads.

    Parameters
    ----------
    chain : Selection
        MDAnalysis Selection object

    Returns
    -------
    s_cc : numpy.ndarray
        Mean S_cc parameter for the selected chain of each residue
    """

    #Separate the coordinates according to their residue index
    ridx = np.where(np.diff(chain.resids) > 0)[0] + 1

    pos = np.split( chain.positions, ridx )
    
    #Calculate the normalized orientation vector between two subsequent tail beads
    vec = [ np.diff( pos_i, axis = 0) for pos_i in pos ]

    vec_norm = [ np.sqrt((vec_i**2).sum( axis = -1 )) for vec_i in vec ]

    vec = [ vec_i / vec_norm_i.reshape( -1, 1) for vec_i, vec_norm_i in zip( vec, vec_norm) ]

    #Choose the z-axis as membrane normal and take care of the machine precision
    dot_prod = [ np.clip(vec_i[:, 2], -1., 1.) for vec_i in vec]

    #Calculate the order parameter
    s_cc = np.array( [ np.mean( 0.5 * ( 3 * dot**2 - 1) ) for dot in dot_prod ] )

    return s_cc

def weight_matrix(vor, coor_xy, bx, by):

    """
    Calculate the weight factors between neighbored lipid pairs based on a Voronoi tessellation.

    Parameters
    ----------
    vor : Voronoi Tesselation
        Scipy's Voronoi Diagram object
    coor_xy : numpy.ndarray
        Mapped positions of the lipid headgroups
    bx : float
        Length of box vector in x direction
    by : float
        Length of box vector in y direction

    Returns
    -------
    weight_matrix : numpy.ndarray
        Weight factors wij between neighbored lipid pairs. 
        Is 0 if lipids are not directly neighbored.
    """

    #Number of points in the plane
    ncoor = coor_xy.shape[0]

    #Calculate the distance for all pairs of points between which a ridge exists
    dij = vor.points[vor.ridge_points[:, 0]] - vor.points[vor.ridge_points[:, 1]]
    dij = np.sqrt( np.sum( dij**2, axis = 1) )

    #There is the (rare!) possibility that two points have the exact same xy positions,
    #to prevent issues at further calculation steps, their distance is set to a very small
    #distance of 4.7 Angstrom (2 times the VdW radius of a regular bead in MARTINI3)
    dij[ dij < 1E-5 ] = 4.7

    #Calculate the distance for all pairs of vertices connected via a ridge
    vert_idx = np.array(vor.ridge_vertices)
    bij = vor.vertices[vert_idx[:, 0]] - vor.vertices[vert_idx[:, 1]]
    bij = np.sqrt( np.sum( bij**2, axis = 1) )

    #INFO: vor.ridge_points and vor.ridge_vertices should be sorted -> Check vor.ridge_dict

    #Calculate weight factor
    wij = bij / dij 

    #Setup an empty array to store the weight factors for each lipid
    weight_matrix = np.zeros( (ncoor, ncoor) )

    #Select all indices of ridges that contain members of the unit cell
    mask_unit_cell = np.logical_or(vor.ridge_points[:, 0] < ncoor, vor.ridge_points[:, 1] < ncoor)

    #Apply the modulus operator since some of the indices in "unit_cell_point" will point to coordinates
    #outside the unit cell. Applying the modulus operator "%" will allow an indexing of the "weight_matrix".
    #However, some of the indices in "unit_cell_point" will be doubled that shouldn't be an issue since the
    #same weight factor is then just put several times in the same entry of the array (no summing or something similar!)
    unit_cell_point=vor.ridge_points[mask_unit_cell] % ncoor

    weight_matrix[ unit_cell_point[:, 0], unit_cell_point[:, 1]] = wij[mask_unit_cell]
    weight_matrix[ unit_cell_point[:, 1], unit_cell_point[:, 0]] = wij[mask_unit_cell]

    return weight_matrix

def area_per_lipid_vor(coor_xy, bx, by):

    """
    Calculation of the area per lipid employing Voronoi tessellation on coordinates mapped to the xy plane.
    The function takes also the periodic boundary conditions of the box into account.

    Parameters
    ----------
    coor_xy : numpy.ndarray
        Mapped positions of the lipid headgroups
    bx : float
        Length of box vector in x direction
    by : float
        Length of box vector in y direction

    Returns
    -------
    apl : numpy.ndarray
        Area per lipid for each residue.
    vor : Voronoi Tesselation
        Scipy's Voronoi Diagram object 
    """

    #Number of points in the plane
    ncoor = coor_xy.shape[0]

    #Create periodic images of the coordinates
    #to take periodic boundary conditions into account
    pbc = np.zeros((9*ncoor,2), dtype=np.float32)

    #Iterate over all possible periodic images
    k = 0
    for i in [0, -1, 1]:
        for j in [0, -1, 1]:

            #Multiply the coordinates in a direction
            pbc[ k*ncoor : (k+1)*ncoor, 0] = coor_xy[:, 0] % bx + i * bx
            pbc[ k*ncoor : (k+1)*ncoor, 1] = coor_xy[:, 1] % by + j * by

            k += 1

    #Call scipy's Voronoi implementation
    #There is the (rare!) possibility that two points have the exact same xy positions,
    #to prevent issues at further calculation steps, the qhull_option "QJ" was employed to introduce small random
    #displacement of the points to resolve these issue.
    vor = Voronoi(pbc, qhull_options="QJ")

    #Iterate over all members of the unit cell and calculate their occupied area
    apl = np.array([ ConvexHull( vor.vertices[vor.regions[vor.point_region[i]]]) .volume for i in range(ncoor) ])


    return apl, vor

## Iteration over the Trajectory

In [ ]:
#Number of frames for the iteration
n_frames = len( uni.trajectory )

#Initialize an array to store the calculated lipid properties for each lipid at each time step
#An artifically large value of 1000 is subtracted to find eventual flaws in the data at later analysis steps
data = np.zeros( (2178, n_frames, 4), dtype = np.float32 ) - 1000

#Due to flip-flops the size of the weight matrices can vary, hence the arrays will be stored in lists
upper_weight_matrix = []
lower_weight_matrix = []

#Store the assignment to a leaflet for each lipid at each time step
leaflet_assignment = np.zeros( (2178, n_frames), dtype = np.int32 )

#Atom selection for order parameter calculation
chainA = uni.select_atoms("name C?A or name D?A")
chainB = uni.select_atoms("name C?B or name D?B")
chainC = uni.select_atoms("name ROH or name C1")

#Atom selection for the whole membrane
membrane = uni.select_atoms("resname POPC PUPC CHOL")

#Atom selection for the head group of cholesterol
cholesterol = uni.select_atoms("name ROH")

#Get the sorted resids for all lipids in the membrane
uniq_resids = uni.select_atoms("name PO4 or name ROH").resids

#Start the loop over the trajectory
for step, ts in tqdm( enumerate( uni.trajectory  ), total = n_frames ):

    #Check the leaflet assignment for each step -> That number can be safely increased to 10 or even 100
    if not step % 1:

        #Assign leaflets based on the center of mass of the membrane
        com_membrane = membrane.center_of_mass()

        #Select phospholipids in the upper and lower leaflet
        upper_leaflet = uni.select_atoms(f"name PO4 and prop z > {com_membrane[2]}")
        lower_leaflet = uni.select_atoms(f"name PO4 and prop z <= {com_membrane[2]}")

    #Assign cholesterol to a leaflet by...

    #...calculating the distance to each lipid headgroup in the lower and the upper leaflet and...
    upper_cholesterol = distances.distance_array(reference = cholesterol, configuration = upper_leaflet, box = ts.dimensions)
    lower_cholesterol = distances.distance_array(reference = cholesterol, configuration = lower_leaflet, box = ts.dimensions)

    #...determining the minimum distance to each leaflet for each cholesterol,...
    upper_cholesterol = np.min(upper_cholesterol, axis = 1)
    lower_cholesterol = np.min(lower_cholesterol, axis = 1)

    #...the assignment is finished by checking for which leaflet the minimum distance is smallest.
    upper_cholesterol = cholesterol[ upper_cholesterol < lower_cholesterol ]
    lower_cholesterol = cholesterol.difference( upper_cholesterol )

    #Merge the atom selections for the phospholipids and cholesterol -> Selection should sorted according to resid
    upper_total_leaflet = upper_leaflet + upper_cholesterol
    lower_total_leaflet = lower_leaflet + lower_cholesterol

    #Resid start at 1, Python indices start at 0.
    uidx = upper_total_leaflet.resids - 1
    lidx = lower_total_leaflet.resids - 1

    #Upper leaflet gets index 0, lower leaflet gets index 1
    leaflet_assignment[uidx, step] = 0
    leaflet_assignment[lidx, step] = 1

    #Area per lipid
    #--------------------------------------------------#

    #Get box vectors
    dim = ts.dimensions[0:3]

    #Calculate the area per for the upper and lower leaflet
    upper_apl, upper_vor = area_per_lipid_vor(coor_xy = upper_total_leaflet.positions[:, 0:2], bx = dim[0], by = dim[1])
    lower_apl, lower_vor= area_per_lipid_vor(coor_xy = lower_total_leaflet.positions[:, 0:2], bx = dim[0], by = dim[1])

    data[uidx, step, 0] = upper_apl
    data[lidx, step, 0] = lower_apl

    #Order Parameters
    #--------------------------------------------------#

    #Order parameters for each selected chain
    s_cc_A = order_parameter(chain = chainA)
    s_cc_B = order_parameter(chain = chainB)
    s_cc_C = order_parameter(chain = chainC) #Cholesterol

    #Store the calculated parameters for each chain.
    #Be aware that np.unique sorts its output!

    _, aidx, _ = np.intersect1d(uniq_resids, np.unique(chainA.resids), return_indices=1)
    data[aidx, step, 1] = s_cc_A

    _, bidx, _ = np.intersect1d(uniq_resids, np.unique(chainB.resids), return_indices=1)
    data[bidx, step, 2] = s_cc_B

    _, cidx, _ = np.intersect1d(uniq_resids, np.unique(chainC.resids), return_indices=1)
    data[cidx, step, 3] = s_cc_C

    #Weight matrix
    #--------------------------------------------------#

    #Calcuate weight matrix based on the same Voronoi diagram that was already used for the area per lipid calculation
    upper_weight_matrix.append( weight_matrix(vor = upper_vor, coor_xy = upper_total_leaflet.positions[:, 0:2], bx = dim[0], by = dim[1]) )
    lower_weight_matrix.append( weight_matrix(vor = lower_vor, coor_xy = lower_total_leaflet.positions[:, 0:2], bx = dim[0], by = dim[1]) )


In [ ]:
#Convert list of weight matrices to numpy array
upper_weight_matrix = np.array(upper_weight_matrix, dtype=object)
lower_weight_matrix = np.array(lower_weight_matrix, dtype=object)

Store the calculated lipid properties at the disk!

In [ ]:
np.save(arr = data, file = "DomHMM_B_Data.npy")

np.save(arr = upper_weight_matrix, file = "DomHMM_B_UpWM.npy", allow_pickle=True)

np.save(arr = lower_weight_matrix, file = "DomHMM_B_LoWM.npy", allow_pickle=True)

np.save(arr = leaflet_assignment , file = "DomHMM_B_LeafletAssign.npy")

In [ ]:
data = np.load("DomHMM_B_Data.npy")

upper_weight_matrix = np.load("DomHMM_B_UpWM.npy", allow_pickle=True)

lower_weight_matrix = np.load("DomHMM_B_LoWM.npy", allow_pickle=True)

leaflet_assignment = np.load("DomHMM_B_LeafletAssign.npy")

## Visualization of Structural Properties

The following code cells show how to select and visualize the above calculated properties.

In [ ]:
resids_POPC = uni.select_atoms("name PO4 and resname POPC").resids
resids_PUPC = uni.select_atoms("name PO4 and resname PUPC").resids
resids_CHOL = uni.select_atoms("name ROH and resname CHOL").resids

_, idxPOPC, _ = np.intersect1d( uniq_resids, resids_POPC, return_indices= 1)
_, idxPUPC, _ = np.intersect1d( uniq_resids, resids_PUPC, return_indices= 1)
_, idxCHOL, _ = np.intersect1d( uniq_resids, resids_CHOL, return_indices= 1)

In [ ]:
#Select the data for each type of lipid (2 columns for cholesterol, 3 columns for phospholipids)
#Check if the ranges of the calculated data are rational
#If something went wrong in the indexing the above subtracted value of 100 should show up
print("Minimum")
print(data[idxCHOL][:, :, [0, 3]].min())
print(data[idxPOPC][:, :, [0, 1,2]].min())
print(data[idxPUPC][:, :, [0, 1,2]].min())
print()

print("Maximum")
print(data[idxCHOL][:, :, [0, 3]].max())
print(data[idxPOPC][:, :, [0, 1,2]].max())
print(data[idxPUPC][:, :, [0, 1,2]].max())

In [ ]:
cm = 1/2.54
fig, ax = plt.subplots(1, 4, figsize=(35*cm, 10*cm))

#Histograms of the area per lipid
#----------------------------------------------------------------------------------------------------------------#
ax[0].hist(data[idxPOPC, :, 0].flatten(), density = True, bins = np.linspace(0, 200, 101), histtype = "step", color = "red", lw=2)
ax[0].hist(data[idxPUPC, :, 0].flatten(), density = True, bins = np.linspace(0, 200, 101), histtype = "step", color = "darkblue", lw=2)
ax[0].hist(data[idxCHOL, :, 0][data[idxCHOL, :, 0]>-1].flatten(), density = True, bins = np.linspace(0, 200, 101), histtype = "step", color = "black", lw=2)
ax[0].legend()

ax[0].set_ylabel(r"$p(a)$", fontsize = 18)
ax[0].set_xlabel(r"$a~(\AA^2)$", fontsize = 18)
ax[0].set_ylim(0, 0.04)
ax[0].set_yticks(np.linspace(0, 0.04, 5))

ax[0].plot([], color = "red", label = "POPC")
ax[0].plot([], color = "darkblue", label = "PUPC")
ax[0].plot([], color = "k", label = "CHOL")
ax[0].legend(loc = "upper right", fontsize = 15)

#Histograms of the order parameters for chain B (sn-1)
#----------------------------------------------------------------------------------------------------------------#
ax[1].hist(data[idxPOPC, :, 2].flatten(), density = True, bins = np.linspace(-.5, 1, 51), histtype = "step", color = "red", lw=2)
ax[1].hist(data[idxPUPC, :, 2].flatten(), density = True, bins = np.linspace(-.5, 1, 51), histtype = "step", color = "darkblue", lw=2)
ax[1].legend()

ax[1].set_ylabel(r"$p(\bar{S}^{\text{sn-1}}_{CC})$", fontsize = 18)
ax[1].set_xlabel(r"$\bar{S}^{\text{sn-1}}_{CC}$", fontsize = 18)
ax[1].set_ylim(0, 3.0)
ax[1].set_xlim(-0.6, 1.1)
ax[1].set_xticks([-0.5, 0.0, 0.5, 1.0])
ax[1].set_yticks(np.linspace(0, 3.0, 4))

ax[1].plot([], color = "red", label = "POPC")
ax[1].plot([], color = "darkblue", label = "PUPC")
ax[1].legend(loc = "upper left", fontsize = 15)

#Histograms of the order parameters for chain A (sn-2)
#----------------------------------------------------------------------------------------------------------------#
ax[2].hist(data[idxPOPC, :, 1].flatten(), density = True, bins = np.linspace(-.5, 1, 51), histtype = "step", color = "red", lw=2)
ax[2].hist(data[idxPUPC, :, 1].flatten(), density = True, bins = np.linspace(-.5, 1, 51), histtype = "step", color = "darkblue", lw=2)
ax[2].legend()

ax[2].set_ylabel(r"$p(\bar{S}^{\text{sn-2}}_{CC})$", fontsize = 18)
ax[2].set_xlabel(r"$\bar{S}^{\text{sn-2}}_{CC}$", fontsize = 18)
ax[2].set_ylim(0, 3.0)
ax[2].set_xlim(-0.6, 1.1)
ax[2].set_xticks([-0.5, 0.0, 0.5, 1.0])
ax[2].set_yticks(np.linspace(0, 3.0, 4))

ax[2].plot([], color = "red", label = "POPC")
ax[2].plot([], color = "darkblue", label = "PUPC")
ax[2].legend(loc = "upper left", fontsize = 15)

#Histograms of the order parameters for cholesterol
#----------------------------------------------------------------------------------------------------------------#
ax[3].hist(data[idxCHOL, :, 3].flatten(), density = True, bins = np.linspace(-.5, 1, 51), histtype = "step", color = "black", lw=2)
ax[3].legend()

ax[3].set_ylabel(r"$p(P_2)$", fontsize = 18)
ax[3].set_xlabel(r"$P_2$", fontsize = 18)
ax[3].set_xlim(-0.6, 1.1)
ax[3].set_xticks([-0.5, 0.0, 0.5, 1.0])
ax[3].set_yticks(np.linspace(0, 7.0, 8))
ax[3].set_ylim(0, 7.0)

ax[3].plot([], color = "k", label = "CHOL")
ax[3].legend(loc = "upper left", fontsize = 15)
    

ax[0].set_title("e", loc = "left", fontweight = "bold", fontsize = 20)
ax[1].set_title("f", loc = "left", fontweight = "bold", fontsize = 20)
ax[2].set_title("g", loc = "left", fontweight = "bold", fontsize = 20)
ax[3].set_title("h", loc = "left", fontweight = "bold", fontsize = 20)

for i in range(3):
    ax[i].tick_params(labelsize=11)

plt.tight_layout()

plt.savefig( "POPC_PUPC_Struc_Prop_CHOL.pdf", dpi=300, transparent=True)

## Gaussian Mixture Model

Initial and fit for each lipid type a Gaussian Mixture model using two components.

In [ ]:
gmm_kwargs={"tol":1E-4, "init_params":'k-means++', "verbose":0,
            "max_iter":10000, "n_init":20,
            "warm_start":False,"covariance_type":"full"}

gmmPOPC = mixture.GaussianMixture( n_components = 2, **gmm_kwargs).fit( data[idxPOPC][:, :, [0, 1, 2]].reshape(-1, 3) )
gmmPUPC = mixture.GaussianMixture( n_components = 2, **gmm_kwargs).fit( data[idxPUPC][:, :, [0, 1, 2]].reshape(-1, 3) )
gmmCHOL = mixture.GaussianMixture( n_components = 2, **gmm_kwargs).fit( data[idxCHOL][:, :, [0, 3]].reshape(-1, 2) )


In [ ]:
with open("gmmPOPC_CHOL.pkl", "wb") as fp: pickle.dump(gmmPOPC, fp)
with open("gmmPUPC_CHOL.pkl", "wb") as fp: pickle.dump(gmmPUPC, fp)
with open("gmmCHOL_B.pkl", "wb") as fp: pickle.dump(gmmCHOL, fp)

In [ ]:
cm =1/2.54
fig, ax = plt.subplots(3, 3, figsize = (35*cm, 28*cm))

#Define parameters that define a grid for the calculations of the fitted distributions
x01, y01 = np.mgrid[0:200:.5, -0.5:1.05:.05]
x2, y2 = np.mgrid[-0.5:1.05:.05, -0.5:1.05:.05]

xy01 = np.dstack((x01, y01))
xy2 = np.dstack((x2, y2))

#Define colors for the markers that mark the fitted means of the Gaussians
cx = ["crimson","cyan"]

#Joint distributions for POPC
#-------------------------------------------------------------------------

#Area per Lipid - Scc Chain A
ax[0, 0].hist2d(x = data[idxPOPC][:, :, [0, 1, 2]].reshape(-1, 3)[:, 0],
                y = data[idxPOPC][:, :, [0, 1, 2]].reshape(-1, 3)[:, 1],
                bins = [np.linspace(0, 200, 101), np.linspace(-.5, 1, 51)],
                density = True, cmap="viridis")

#Area per Lipid - Scc Chain B
ax[0, 1].hist2d(x = data[idxPOPC][:, :, [0, 1, 2]].reshape(-1, 3)[:, 0],
                y = data[idxPOPC][:, :, [0, 1, 2]].reshape(-1, 3)[:, 2],
                bins = [np.linspace(0, 200, 101), np.linspace(-.5, 1, 51)],
                density = True, cmap="viridis")

#Scc Chain A - Scc Chain B
ax[0, 2].hist2d(x = data[idxPOPC][:, :, [0, 1, 2]].reshape(-1, 3)[:, 1],
                y = data[idxPOPC][:, :, [0, 1, 2]].reshape(-1, 3)[:, 2],
                bins = [np.linspace(-.5, 1, 51), np.linspace(-.5, 1, 51)],
                density = True, cmap="viridis")

#Calculate and display fitted distributions
#Iterate over both Gaussian distributions
for i in range(2):

    #Area per Lipid (0) - Scc Chain A (1)
    rv0 = stats.multivariate_normal(gmmPOPC.means_[i][0:2], gmmPOPC.covariances_[i][:2,:2])
    z0 = rv0.pdf(xy01)

    #Area per Lipid (0) - Scc Chain B (2)
    rv1 = stats.multivariate_normal(gmmPOPC.means_[i][::2], gmmPOPC.covariances_[i][::2, ::2])
    z1 = rv1.pdf(xy01)

    #Scc Chain A (1) - Scc Chain B (2)
    rv2 = stats.multivariate_normal(gmmPOPC.means_[i][1:], gmmPOPC.covariances_[i][1:, 1:])
    z2 = rv2.pdf(xy2)

    #Plot contours -> Be aware that the first two plots share the same ranges
    ax[0, 0].contour(x01, y01, z0, cmap = "coolwarm", alpha = 0.5)
    ax[0, 1].contour(x01, y01, z1, cmap = "coolwarm", alpha = 0.5)
    ax[0, 2].contour(x2, y2, z2, cmap = "coolwarm", alpha = 0.5)

    #Mark with a cross the means of the fitted Gaussians
    ax[0,0].scatter( gmmPOPC.means_[i][0], gmmPOPC.means_[i][1], marker = "x", s = 100, zorder = 100, color=cx[i])
    ax[0,1].scatter( gmmPOPC.means_[i][0], gmmPOPC.means_[i][2], marker = "x", s = 100, zorder = 100, color=cx[i])
    ax[0,2].scatter( gmmPOPC.means_[i][1], gmmPOPC.means_[i][2], marker = "x", s = 100, zorder = 100, color=cx[i])

#Joint distributions for PUPC
#-------------------------------------------------------------------------

#Area per Lipid - Scc Chain A
ax[1, 0].hist2d(x = data[idxPUPC][:, :, [0, 1, 2]].reshape(-1, 3)[:, 0],
                y = data[idxPUPC][:, :, [0, 1, 2]].reshape(-1, 3)[:, 1],
                bins = [np.linspace(0, 200, 101), np.linspace(-.5, 1, 51)],
                density = True, cmap="viridis")

#Area per Lipid - Scc Chain B
ax[1, 1].hist2d(x = data[idxPUPC][:, :, [0, 1, 2]].reshape(-1, 3)[:, 0],
                y = data[idxPUPC][:, :, [0, 1, 2]].reshape(-1, 3)[:, 2],
                bins = [np.linspace(0, 200, 101), np.linspace(-.5, 1, 51)],
                density = True, cmap="viridis")

#Scc Chain A - Scc Chain B
ax[1, 2].hist2d(x = data[idxPUPC][:, :, [0, 1, 2]].reshape(-1, 3)[:, 1],
                y = data[idxPUPC][:, :, [0, 1, 2]].reshape(-1, 3)[:, 2],
                bins = [np.linspace(-.5, 1, 51), np.linspace(-.5, 1, 51)],
                density = True, cmap="viridis")

#Calculate and display fitted distributions
#Iterate over both Gaussian distributions
for i in range(2):

    #Area per Lipid (0) - Scc Chain A (1)
    rv0 = stats.multivariate_normal(gmmPUPC.means_[i][0:2], gmmPUPC.covariances_[i][:2,:2])
    z0 = rv0.pdf(xy01)
    
    #Area per Lipid (0) - Scc Chain B (2)
    rv1 = stats.multivariate_normal(gmmPUPC.means_[i][::2], gmmPUPC.covariances_[i][::2, ::2])
    z1 = rv1.pdf(xy01)

    #Scc Chain A (1) - Scc Chain B (2)
    rv2 = stats.multivariate_normal(gmmPUPC.means_[i][1:], gmmPUPC.covariances_[i][1:, 1:])
    z2 = rv2.pdf(xy2)

    #Plot contours -> Be aware that the first two plots share the same ranges
    ax[1, 0].contour(x01, y01, z0, cmap = "coolwarm", alpha = 0.5)
    ax[1, 1].contour(x01, y01, z1, cmap = "coolwarm", alpha = 0.5)
    ax[1, 2].contour(x2, y2, z2, cmap = "coolwarm", alpha = 0.5)

    #Mark with a cross the means of the fitted Gaussians
    ax[1,0].scatter( gmmPUPC.means_[i][0], gmmPUPC.means_[i][1], marker = "x", s = 100, zorder = 100, color=cx[i])
    ax[1,1].scatter( gmmPUPC.means_[i][0], gmmPUPC.means_[i][2], marker = "x", s = 100, zorder = 100, color=cx[i])
    ax[1,2].scatter( gmmPUPC.means_[i][1], gmmPUPC.means_[i][2], marker = "x", s = 100, zorder = 100, color=cx[i])

#Joint distributions for Cholesterol
#-------------------------------------------------------------------------

ax[2, 1].hist2d(x = data[idxCHOL][:, :, [0, 3]].reshape(-1, 2)[:, 0],
                y = data[idxCHOL][:, :, [0, 3]].reshape(-1, 2)[:, 1],
                bins = [np.linspace(0, 200, 101), np.linspace(-.5, 1, 51)],
                density = True, cmap="viridis")

#Calculate and display fitted distributions
#Iterate over both Gaussian distributions
for i in range(2):

    #Area per Lipid (0) - Scc Chain C (1)
    rv0 = stats.multivariate_normal(gmmCHOL.means_[i], gmmCHOL.covariances_[i])
    z0 = rv0.pdf(xy01)

    #Plot contours
    ax[2, 1].contour(x01, y01, z0, cmap = "coolwarm", alpha = 0.5)

    #Mark with a cross the means of the fitted Gaussians
    ax[2,1].scatter( gmmCHOL.means_[i][0], gmmCHOL.means_[i][1], marker = "x", s = 100, zorder = 100, color=cx[i])

for i in range(3): ax[0, i].set_xticks([])
ax[1, 1].set_xticks([])

for i in range(3):
    ax[0,i].set_yticks([-0.5, 0, 0.5, 1], [r"$-0.5$", r"$0$", r"$0.5$", r"$1$"])
    ax[1,i].set_yticks([-0.5, 0, 0.5, 1], [r"$-0.5$", r"$0$", r"$0.5$", r"$1$"])
    ax[2,i].set_yticks([-0.5, 0, 0.5, 1], [r"$-0.5$", r"$0$", r"$0.5$", r"$1$"])


ax[1,0].set_xticks([0, 50, 100, 150, 200])
ax[2,1].set_xticks([0, 50, 100, 150, 200])
ax[1,2].set_xticks([-0.5, 0, 0.5, 1], [r"$-0.5$", r"$0$", r"$0.5$", r"$1$"])

for i in range(2):
    ax[i,0].set_ylabel(r"$p(\bar{S}_{CC}^{\text{sn-2}})$", labelpad=-7,fontsize=18)
    ax[i,1].set_ylabel(r"$p(\bar{S}_{CC}^{\text{sn-1}})$", labelpad=-7,fontsize=18)
    ax[i,2].set_ylabel(r"$p(\bar{S}_{CC}^{\text{sn-1}})$", labelpad=-7,fontsize=18)
ax[2,1].set_ylabel(r"$p(P_2)$", labelpad=-7,fontsize=18)

ax[1,0].set_xlabel(r"$p(a)$", labelpad=0,fontsize=18)
ax[2,1].set_xlabel(r"$p(a)$", labelpad=0,fontsize=18)
ax[1,2].set_xlabel(r"$p(\bar{S}_{CC}^{\text{sn-2}})$", labelpad=0,fontsize=18)


ax[0,0].set_title(r"a", fontsize=20, fontweight="bold", loc = "left")
ax[0,1].set_title(r"b", fontsize=20, fontweight="bold", loc = "left")
ax[0,2].set_title(r"c", fontsize=20, fontweight="bold", loc = "left")

for i in range(3):
    for j in range(3):
        ax[i, j].tick_params(rotation=45)
        ax[i, j].grid(False)
        ax[i, j].tick_params(axis="both", labelsize=11)

plt.subplots_adjust(hspace=0.1, wspace = 0.25)

ax[0,0].text(s = "POPC", x= -70, y= 0.25, rotation=90, ha = "center", va = "center",fontsize=20, fontweight="bold")
ax[1,0].text(s = "PUPC", x= -70, y= 0.25, rotation=90, ha = "center", va = "center",fontsize=20, fontweight="bold")
ax[2,1].text(s = "CHOL", x= -318, y= 0.25, rotation=90, ha = "center", va = "center",fontsize=20, fontweight="bold")

ax[2,0].set_visible(False)
ax[2,2].set_visible(False)

plt.savefig("GMM_B_CHOL.pdf", dpi = 300, transparent=True)

Check for convergence and singular covariance matrices

In [ ]:
gmmPOPC.converged_, gmmPUPC.converged_, gmmCHOL.converged_

In [ ]:
#None of the determinants should be 0
for i in range(2):
    print(np.linalg.det(gmmPOPC.covariances_[i]))
    print(np.linalg.det(gmmPUPC.covariances_[i]))
    print(np.linalg.det(gmmCHOL.covariances_[i]))

## Gaussian-based Hidden Markov Model

In [ ]:
def fit_hmm( data, gmm, n_repeats = 10, dim = 3):

    """
    Fit several HMM models to the data and return the best one. 

    Parameters
    ----------
    data : numpy.ndarray
        Data of the single lipid properties for one lipid type at each time step
    gmm : GaussianMixture Model
        Scikit-learn object
    n_repeats : int
        Number of independent fits
    dim : int
        Dimension of lipid property space

    Returns
    -------
    best_ghmm : GaussianHMM
        hmmlearn object

    """

    #Additional parameters for the GaussianHMM
    hmm_kwargs = {"verbose":0, "tol":1E-4,"n_iter":2000,
                  "algorithm":"viterbi", "covariance_type":"full",
                  "init_params":"st", "params":"stmc"}

    #Specify the length of the sequence for each lipid
    n_lipids = data.shape[0]
    lengths = np.repeat( 2001, n_lipids )

    #The HMM fitting is started multiple times from
    #different starting conditions

    #Initialize the best score with minus infinity
    best_score = -np.inf

    #Re-start the HMM fitting 10 times
    for i in tqdm(range( n_repeats )):

        #Initialize a HMM for one lipid type
        ghmm_i = GaussianHMM(n_components = 2, 
                             means_prior = gmm.means_,
                             covars_prior = gmm.covariances_,
                             **hmm_kwargs)

        #Train the HMM based on the data for every lipid and frame
        ghmm_i.fit(data.reshape(-1, dim),
                   lengths = lengths )

        #Obtain the log-likelihood
        #probability of the current model
        score_i = ghmm_i.score(data.reshape(-1, dim),
                               lengths = lengths )

        #Check if the quality of the result improved
        if score_i > best_score:
            best_score = score_i
            best_ghmm = ghmm_i

        #Delete the current model
        del ghmm_i
        
    return best_ghmm

In [ ]:
ghmmPOPC = fit_hmm( data = data[idxPOPC][:, :, [0, 1, 2]], gmm = gmmPOPC, n_repeats = 10)
ghmmPUPC = fit_hmm( data = data[idxPUPC][:, :, [0, 1, 2]], gmm = gmmPUPC, n_repeats = 10)

In [ ]:
ghmmCHOL = fit_hmm( data = data[idxCHOL][:, :, [0, 3]], gmm = gmmCHOL, n_repeats = 10, ndim = 2)

Store the trained models as pickled files.

In [ ]:
with open("ghmmPOPC_CHOL.pkl", "wb") as fp: pickle.dump(gmmPOPC, fp)
with open("ghmmPUPC_CHOL.pkl", "wb") as fp: pickle.dump(gmmPUPC, fp)
with open("ghmmCHOL_B.pkl", "wb") as fp: pickle.dump(gmmCHOL, fp)

In [ ]:
#Plot the changes in the log-likelihood of the models
plt.semilogy(np.arange(len(ghmmPOPC.monitor_.history)-1), np.diff(np.array(ghmmPOPC.monitor_.history)), color ="r", ls="-", label="POPC")
plt.semilogy(np.arange(len(ghmmPUPC.monitor_.history)-1), np.diff(np.array(ghmmPUPC.monitor_.history)), color ="darkblue", ls="-", label="PUPC")
plt.semilogy(np.arange(len(ghmmCHOL.monitor_.history)-1), np.diff(np.array(ghmmCHOL.monitor_.history)), color ="k", ls="-", label="CHOL")
plt.legend(fontsize=15)
plt.semilogy(np.arange(500), np.repeat(1E-4, 500), color ="k", ls="--")
plt.xlim(0, 500)
plt.ylim(1E-5, 15E5)
plt.ylabel(r"$\Delta(log(\hat{L}))$",fontsize=18)
plt.xlabel("Iterations", fontsize=18)
plt.tick_params(axis="both", labelsize=11)
plt.text(s = "Tolerance 1e-4", x = 90, y = 1E-4+0.00005, color = "k", fontsize = 15)
plt.title("d", fontsize=20, fontweight="bold", loc = "left")
plt.savefig("GHMMConv_B.pdf", transparent=True, dpi=300)

The next code cells predict the order states $[O_o, O_d]$ with the previously fitted models.

<div class="alert alert-danger">

Be aware that the predictions of the order states must not be consistent between the individual HMMs.
For a comparison and not running into troubles at later steps of the workflow, the means of the states could checked.
State 0 should be the disordered state $O_d$, showing therefore a high area per lipid and lower order parameters. Vice versa,
State 1 should be the ordered state $O_o$, showing therefore a lower area per lipid and higher order parameters.
</div>

Print and compare the means. Higher area per Lipids and lower order parameters must be at row 0 (disordered), while lower area per lipids and higher order parameters must be at row 1 (ordered). If it is the higher way around, the states must be re-labelled!

In [ ]:
ghmmPUPC.means_

In [ ]:
ghmmPOPC.means_

In [ ]:
ghmmCHOL.means_

In [ ]:
#Get the lengths of each temporal sequence
lengthsPOPC = np.repeat(2001, 726)
lengthsPUPC = np.repeat(2001, 726)
lengthsCHOL = np.repeat(2001, 726)

In [ ]:
predictPOPC=ghmmPOPC.predict( data[idxPOPC][:, :, [0, 1, 2]].reshape(-1,3), lengths = lengthsPOPC).reshape(726, 2001)
predictPUPC=ghmmPUPC.predict( data[idxPUPC][:, :, [0, 1, 2]].reshape(-1,3), lengths = lengthsPUPC).reshape(726, 2001)
predictCHOL=ghmmCHOL.predict( data[idxCHOL][:, :, [0, 3]].reshape(-1,2), lengths = lengthsCHOL).reshape(726, 2001)

#For the current set of GHMMs the predicted labels of the following lipids must be flipped
predictPUPC=np.abs(predictPUPC-1)
predictPOPC=np.abs(predictPOPC-1)
predictCHOL=np.abs(predictCHOL-1)

In [ ]:
#Plot the average predictions at each time step for each type of lipid
t = np.linspace(8, 10, 2001)
im = plt.plot(t,predictPOPC.mean(0), color = "red",label = "POPC")
im = plt.plot(t,predictPUPC.mean(0), color = "darkblue",label = "PUPC")
im = plt.plot(t,predictCHOL.mean(0), color = "k",label = "CHOL")

plt.xticks([8, 8.5, 9, 9.5, 10])

plt.xlabel(r"t ($\mu$s)", fontsize=18)
plt.ylabel(r"$\bar{O}_{Lipid}$", fontsize=18)

plt.legend(fontsize=15, ncols=1, loc="lower left")

plt.ylim(0, 1)
plt.xlim(8,10)
plt.title("e", fontsize=20, fontweight="bold", loc = "left")
plt.savefig("GHMMPred_B_CHOL.pdf", transparent=True, dpi=300)

## Getis-Ord Local Spatial Autocorrelation Statistics

In [ ]:
def getis_ord_stat(weight_matrix_all, orderPUPC, orderPOPC, orderCHOL, idxPUPC, idxPOPC, idxCHOL, leaflet, lassign):

    """
    Getis-Ord Local Spatial Autocorrelation Statistics calculation based on the predicted order states
    of each lipid and the weighting factors between neighbored lipids.

    Be aware that this function is far from being elegant but it should do its job.

    Parameters
    ----------
    weight_matrix_all : numpy.ndarray
        Weight matrices for all lipid in a leaflet at each time step
    orderPUPC : numpy.ndarray
        Predicted order states for each PUPC at each time step
    orderPOPC : numpy.ndarray
        Predicted order states for each POPC at each time step
    orderCHOL : numpy.ndarray
        Predicted order states for each CHOL at each time step
    idxPUPC : numpy.ndarray
        Indices of PUPC residues
    idxPOPC : numpy.ndarray
        Indices of POPC residues
    idxCHOL : numpy.ndarray
        Indices of cholesterol residues
    leaflet : int
        0 = upper leaflet, 1 = lower leafet
    lassign : numpy.ndarray
        Leaflet assignment for each molecule at each step of time

    Returns
    -------
    g_star_i : list of numpy.ndarrays
        G*i values for each lipid at each time step
    w_ii_all : list of numpy.ndarrays
        Self-influence of the lipids
    """

    #Get the number of frames
    nframes = weight_matrix_all.shape[0]

    #Initialize empty lists to store the G*i values and the self-influence of the lipids in a lipid
    g_star_i = []
    w_ii_all = []
    
    #Iterate over frames
    for step in range( nframes ):

        #Get the weightmatrix of the leaflet at the current time step
        weight_matrix = weight_matrix_all[step]

        #Number of lipids in the leaflet
        n = weight_matrix.shape[0]

        #In case the code was already executed beforehand
        weight_matrix[ range(n), range(n)] = 0.0

        #Get the order state of each lipid in the leaflet at the current time step 
        #1. Step: (lassign[ idxPUPC ][:, step] == leaflet) -> Which lipids are in the leaflet
        #2. Step: orderPUPC[:, step][ ... ] --> What are their order parameters
        do_pupc = orderPUPC[:, step][ lassign[ idxPUPC ][:, step] == leaflet ]
        do_popc = orderPOPC[:, step][ lassign[ idxPOPC ][:, step] == leaflet ]
        do_chol = orderCHOL[:, step][ lassign[ idxCHOL ][:, step] == leaflet ]

        #Put all the order states in one array -> The order of the lipids must be the same as in the system!!!
        order_states = np.concatenate([do_pupc, do_popc, do_chol])

        #Number of neighbors per lipid -> The number is 0 (or close to 0) for not neighboured lipids
        nneighbor = np.sum(weight_matrix > 1E-5, axis = 1)
        
        #Parameters for the Getis-Ord statistic
        w_ii      = np.sum(weight_matrix, axis = -1)    / nneighbor #Self-influence!
        weight_matrix[ range(n), range(n)] = w_ii
        w_star_i  = np.sum(weight_matrix, axis = -1)    #+ w_ii
        s_star_1i = np.sum(weight_matrix**2, axis = -1) #+ w_ii**2
       
        #Empirical standard deviation over all order states in the leaflet
        s = np.std( order_states, ddof = 1)

        #Employ matrix-vector multiplication
        so = weight_matrix @ order_states

        #Calculate the nominator for G*i
        nom = so  - w_star_i * 0.5

        #Calculate the denominator for G*i
        denom = s * np.sqrt( (n * s_star_1i - w_star_i**2)/(n - 1) )

        g_star = nom / denom

        assert not np.any(nneighbor < 1), "Lipid found without a neighbor!"
        
        g_star_i.append( g_star )
        w_ii_all.append(w_ii)

    return g_star_i, w_ii_all

g_star_i_0, w_ii_0 = getis_ord_stat(weight_matrix_all=upper_weight_matrix,
                                    orderPUPC=predictPUPC, idxPUPC=idxPUPC,
                                    orderPOPC=predictPOPC, idxPOPC=idxPOPC,
                                    orderCHOL=predictCHOL, idxCHOL=idxCHOL,
                                    leaflet=0,
                                    lassign=leaflet_assignment)

g_star_i_1, w_ii_1 = getis_ord_stat(weight_matrix_all=lower_weight_matrix,
                                    orderPUPC=predictPUPC, idxPUPC=idxPUPC,
                                    orderPOPC=predictPOPC, idxPOPC=idxPOPC,
                                    orderCHOL=predictCHOL, idxCHOL=idxCHOL,
                                    leaflet=1,
                                    lassign=leaflet_assignment)

In [ ]:
g_star_i_pupc = []
g_star_i_popc = []
g_star_i_chol = []

for step in range(2001):

    PUPCi0 = np.where(leaflet_assignment[ idxPUPC ][:, step]==0)[0].shape[0]
    POPCi0 = np.where(leaflet_assignment[ idxPOPC ][:, step]==0)[0].shape[0] + PUPCi0

    PUPCi1 = np.where(leaflet_assignment[ idxPUPC ][:, step]==1)[0].shape[0]
    POPCi1 = np.where(leaflet_assignment[ idxPOPC ][:, step]==1)[0].shape[0] + PUPCi1

    g_star_i_pupc += list( np.append(g_star_i_0[step][:PUPCi0], g_star_i_1[step][:PUPCi1]))
    g_star_i_popc += list( np.append(g_star_i_0[step][PUPCi0:POPCi0], g_star_i_1[step][PUPCi1:POPCi1]))
    g_star_i_chol += list( np.append(g_star_i_0[step][POPCi0:], g_star_i_1[step][POPCi1:]))

In [ ]:
def permut_getis_ord_stat(weight_matrix_all, orderPUPC, orderPOPC, orderCHOL, idxPUPC, idxPOPC, idxCHOL, leaflet, lassign):

    """
    Getis-Ord Local Spatial Autocorrelation Statistics calculation based on the predicted order states
    of each lipid and the weighting factors between neighbored lipids.

    Be aware that this function is far from being elegant but it should do its job.

    Parameters
    ----------
    weight_matrix_all : numpy.ndarray
        Weight matrices for all lipid in a leaflet at each time step
    orderPUPC : numpy.ndarray
        Predicted order states for each PUPC at each time step
    orderPOPC : numpy.ndarray
        Predicted order states for each POPC at each time step
    orderCHOL : numpy.ndarray
        Predicted order states for each CHOL at each time step
    idxPUPC : numpy.ndarray
        Indices of PUPC residues
    idxPOPC : numpy.ndarray
        Indices of POPC residues
    idxCHOL : numpy.ndarray
        Indices of cholesterol residues
    leaflet : int
        0 = upper leaflet, 1 = lower leafet
    lassign : numpy.ndarray
        Leaflet assignment for each molecule at each step of time

    Returns
    -------
    g_star_i : list of numpy.ndarrays
        G*i values for each lipid at each time step
    """

    #Get the number of frames
    nframes = weight_matrix_all.shape[0]

    #Initialize empty lists to store the G*i values and the self-influence of the lipids in a lipid
    g_star_i = []
    w_ii_all = []

    #Do 10 permutations per time step
    n_permut=10
    
    #Iterate over frames
    for step in tqdm(range( nframes )):

        for permut in range(n_permut):

            #Get the weightmatrix of the leaflet at the current time step
            weight_matrix = weight_matrix_all[step]
    
            #Number of lipids in the leaflet
            n = weight_matrix.shape[0]
    
            #In case the code was already executed beforehand
            weight_matrix[ range(n), range(n)] = 0.0
    
            #Get the order state of each lipid in the leaflet at the current time step 
            #1. Step: (lassign[ idxPUPC ][:, step] == leaflet) -> Which lipids are in the leaflet
            #2. Step: orderPUPC[:, step][ ... ] --> What are their order parameters
            do_pupc = orderPUPC[:, step][ lassign[ idxPUPC ][:, step] == leaflet ]
            do_popc = orderPOPC[:, step][ lassign[ idxPOPC ][:, step] == leaflet ]
            do_chol = orderCHOL[:, step][ lassign[ idxCHOL ][:, step] == leaflet ]
    
            #Put all the order states in one array -> The order of the lipids must be the same as in the system!!!
            order_states = np.concatenate([do_pupc, do_popc, do_chol])

            np.random.shuffle( order_states )
    
            #Number of neighbors per lipid -> The number is 0 (or close to 0) for not neighboured lipids
            nneighbor = np.sum(weight_matrix > 1E-5, axis = 1)
            
            #Parameters for the Getis-Ord statistic
            w_ii      = np.sum(weight_matrix, axis = -1)    / nneighbor #Self-influence!
            weight_matrix[ range(n), range(n)] = w_ii
            w_star_i  = np.sum(weight_matrix, axis = -1)    #+ w_ii
            s_star_1i = np.sum(weight_matrix**2, axis = -1) #+ w_ii**2
           
            #Empirical standard deviation over all order states in the leaflet
            s = np.std( order_states, ddof = 1)
    
            #Employ matrix-vector multiplication
            so = weight_matrix @ order_states
    
            #Calculate the nominator for G*i
            nom = so  - w_star_i * 0.5
    
            #Calculate the denominator for G*i
            denom = s * np.sqrt( (n * s_star_1i - w_star_i**2)/(n - 1) )
    
            g_star = nom / denom
    
            assert not np.any(nneighbor < 1), "Lipid found without a neighbor!"
            
            g_star_i += list( g_star )

    return g_star_i

permut_g_star_i_0 = permut_getis_ord_stat(weight_matrix_all=upper_weight_matrix,
                                    orderPUPC=predictPUPC, idxPUPC=idxPUPC,
                                    orderPOPC=predictPOPC, idxPOPC=idxPOPC,
                                    orderCHOL=predictCHOL, idxCHOL=idxCHOL,
                                    leaflet=0,
                                    lassign=leaflet_assignment)

permut_g_star_i_1 = permut_getis_ord_stat(weight_matrix_all=lower_weight_matrix,
                                    orderPUPC=predictPUPC, idxPUPC=idxPUPC,
                                    orderPOPC=predictPOPC, idxPOPC=idxPOPC,
                                    orderCHOL=predictCHOL, idxCHOL=idxCHOL,
                                    leaflet=1,
                                    lassign=leaflet_assignment)

In [ ]:
#Plot the distritbution of the G*i values for every lipid type
plt.hist(g_star_i_popc, bins = np.linspace(-3, 3, 201), density=True, histtype="step", color = "red", lw=2)
plt.hist(g_star_i_pupc, bins = np.linspace(-3, 3, 201), density=True, histtype="step", color = "darkblue", lw=2)
plt.hist(g_star_i_chol, bins = np.linspace(-3, 3, 201), density=True, histtype="step", color = "k", lw=2)
plt.hist(np.append(permut_g_star_i_0, permut_g_star_i_1),
         bins = np.linspace(-3, 3, 201),
         density=True, histtype="step", color = "darkorange", lw=2)

plt.plot([], color = "red", label = "POPC")
plt.plot([], color = "darkblue", label = "PUPC")
plt.plot([], color = "k", label = "CHOL")
plt.plot([], color = "darkorange", label = r"$G^*_i~(Permut.)$")

plt.legend(fontsize = 15, loc="upper left", ncols=2)

plt.xlim(-3., 3.)
plt.ylim(0, 0.9)

xl=plt.xlabel("$G^*_i$", fontsize = 18)
plt.ylabel("$p(G^*_i)$", fontsize = 18)
plt.tick_params(labelsize=11)

plt.title("c", fontsize=20, fontweight="bold", loc = "left")
plt.savefig("GO_B_CHOL.pdf", transparent=True, dpi=300, bbox_extra_artists=[xl,], bbox_inches="tight")

In [ ]:
num_ord = np.sum(predictPOPC==1) + np.sum(predictPOPC==1) + np.sum(predictCHOL==1)

num_dis = np.sum(predictPUPC==0) + np.sum(predictPUPC==0) + np.sum(predictCHOL==0)

num_total = num_ord + num_dis

print(f"Frac. of 'o' states: {(num_ord / num_total) * 100:3.2f} %")

print(f"Frac. of 'd' states: {(num_dis / num_total) * 100:2.2f} %")

In [ ]:
np.round(np.quantile(np.append(permut_g_star_i_0, permut_g_star_i_1), q=[0.95]), 3)

In [ ]:
np.round(np.quantile(np.append(permut_g_star_i_0, permut_g_star_i_1), q=[0.05]), 3)

## Hierarchical Clustering

## Approach by Park et al.

In [ ]:
def assign_core_lipids(weight_matrix_f, g_star_i_f, order_states_f, w_ii_f):

    """
    Assign lipids as core members (aka lipids with a high positive autocorrelation)
    depending on the Getis-Ord spatial local autocorrelation statistic.

    Parameters
    ----------
    weight_matrix_f : numpy.ndarray 
        Matrix containing the weight factors between all lipid pairs at one time step
    g_star_i_f : numpy.ndarray  
        Getis-Ord spatial local autocorrelation statistic for every lipid at one time step
    order_states_f: numpy.ndarray 
        Order states for every lipid at one time step
    w_ii_f: numpy.ndarray 
        Self-influence weight factor for every lipid at one time step


    Returns
    -------
    core_lipids : numpy.ndarray (bool)
       Contains a TRUE value if the lipid is a core member, otherwise it FALSE
    """

    #Define boundary of the rection region
    z1_a = 1.750
    za = -1.475
    
    #Assign core members according to their auto-correlation
    core_lipids = g_star_i_f > z1_a

    #Assign lipids with a mid-range auto-correlation (-z_1-a, z_1-a) * Ordered
    low_corr = (g_star_i_f <= z1_a) & (g_star_i_f >= za) & (order_states_f == 1)

    #Add iteratively new lipids to the core members
    n_cores_old = np.inf
    n_cores_new = np.sum(core_lipids)

    #Iterate until self-consistency is reached
    while n_cores_old != n_cores_new:

        #Check how tightly the lipids are connected to the core members
        new_core_lipids = (weight_matrix_f[core_lipids].sum(0) > w_ii_f) & low_corr

        #Assign lipids to core members if condition is full-filled
        core_lipids[ new_core_lipids ] = True

        #Update number of core lipids
        n_cores_old = n_cores_new
        n_cores_new = np.sum(core_lipids)

    return core_lipids


def hierarchical_clustering( weight_matrix_f, w_ii_f, core_lipids ):

    """
    Hierarchical clustering approach to identify spatial related Lo domains.

    Parameters
    ----------
    weight_matrix_f : numpy.ndarray
        Matrix containing the weight factors between all lipid pairs at one time step
    w_ii_f: numpy.ndarray
        Self-influence weight factor for every lipid at one time step
    core_lipids : numpy.ndarray
        Array contains information which lipid is assigned as core member

    Returns
    -------
    clusters : dict
        The dictionary contains each found cluster
    
    """
    
    #Merge iteratively clusters
    n_clusters_old = np.inf
    n_clusters_new = np.sum(core_lipids)
    
    #Get the indices of the core lipids
    core_lipids_id = np.where( core_lipids )[0]
    
    #Store clusters in a Python dictionary
    #Initialize all core lipids as clusters
    clusters = dict( 
                    zip( 
                        core_lipids_id.astype("U"),
                        [ [id] for id in core_lipids_id ]
                        )
                   )

    #Iterate until self-consistency is reached
    while n_clusters_old != n_clusters_new:
    
        #Get a list of the IDs of current clusters
        cluster_ids = list( clusters.keys() )
    
        #Iterate over all clusters i
        for i, id_i in enumerate(cluster_ids):
    
            #If cluster i was already merged and deleted, skip it!
            if id_i not in clusters.keys(): continue
    
            #The cluster weights are defined as the sum
            #over the weights of all lipid members
            cluster_weights_i = np.sum( weight_matrix_f[ clusters[id_i] ], axis = 0)
    
            #Compare cluster weights to the self-influence of each lipid
            merge_condition_i = cluster_weights_i > w_ii_f
    
            #Iterate over all clusters j
            for id_j in cluster_ids[(i+1):]:
    
                #Do not merge a cluster with itself
                if id_i == id_j: continue
                #If cluster j was already merged and deleted, skip it!
                if id_j not in clusters.keys(): continue
    
                #Calculate cluster weights and compare to lipids self-influence
                cluster_weights_j = np.sum( weight_matrix_f[ clusters[id_j] ], axis = 0)
                merge_condition_j = cluster_weights_j > w_ii_f
    
                #If the condition is fullfilled for any lipid -> Merge the clusters
                if np.any( merge_condition_i[ clusters[id_j] ] ) or  np.any( merge_condition_j[ clusters[id_i] ] ):
    
                    #Merge cluster j into cluster i
                    clusters[ id_i ] += clusters[id_j]
    
                    #Delete cluster j from the cluster dict
                    del clusters[id_j]
    
        #Update cluster numbers
        n_clusters_old = n_clusters_new
        n_clusters_new = len(clusters.keys())

    return clusters

In [ ]:
#Init figure
iax = plt.figure()

#Select the last frame of the trajectory
i = -1
uni.trajectory[i]

#Leaflet Assignment
#----------------------------------------------------------------------------------------------------------------------
com_membrane = membrane.center_of_mass()

upper_leaflet = uni.select_atoms(f"name PO4 and prop z > {com_membrane[2]}")
lower_leaflet = uni.select_atoms(f"name PO4 and prop z <= {com_membrane[2]}")

upper_cholesterol = distances.distance_array(reference = cholesterol, configuration = upper_leaflet, box = ts.dimensions)
lower_cholesterol = distances.distance_array(reference = cholesterol, configuration = lower_leaflet, box = ts.dimensions)

upper_cholesterol = np.min(upper_cholesterol, axis = 1)
lower_cholesterol = np.min(lower_cholesterol, axis = 1)

upper_cholesterol = cholesterol[ upper_cholesterol < lower_cholesterol ]
lower_cholesterol = cholesterol.difference( upper_cholesterol )

upper_total_leaflet = upper_leaflet + upper_cholesterol
lower_total_leaflet = lower_leaflet + lower_cholesterol

#Get predicted states for each lipid
#----------------------------------------------------------------------------------------------------------------------
do_popc = predictPOPC[:, i][ leaflet_assignment[:, i][ idxPOPC ] == 0 ]
do_pupc = predictPUPC[:, i][ leaflet_assignment[:, i][ idxPUPC ] == 0 ]
do_chol = predictCHOL[:, i][ leaflet_assignment[:, i][ idxCHOL ] == 0 ]

order_states_0 = np.concatenate([do_pupc, do_popc, do_chol])

#Clustering
#----------------------------------------------------------------------------------------------------------------------
core_lipids = assign_core_lipids(weight_matrix_f = upper_weight_matrix[i],
                                 g_star_i_f      = g_star_i_0[i],
                                 order_states_f  = order_states_0,
                                 w_ii_f          = w_ii_0[i])

clusters = hierarchical_clustering(weight_matrix_f = upper_weight_matrix[i],
                                   w_ii_f          = w_ii_0[i], 
                                   core_lipids     = core_lipids)

#Plot coordinates
#----------------------------------------------------------------------------------------------------------------------
PUPCi0 = np.where(leaflet_assignment[ idxPUPC ][:, step]==0)[0].shape[0]
POPCi0 = np.where(leaflet_assignment[ idxPOPC ][:, step]==0)[0].shape[0] + PUPCi0

#Plot PUPC
plt.scatter(upper_total_leaflet.positions[:PUPCi0, 0],
            upper_total_leaflet.positions[:PUPCi0, 1],
            marker = "^", color="darkblue", alpha = 1, s=2)

#Plot POPC
plt.scatter(upper_total_leaflet.positions[PUPCi0:POPCi0, 0],
            upper_total_leaflet.positions[PUPCi0:POPCi0, 1],
            marker = "s", color="red", alpha = 1, s=2)

#Plot Cholesterol
plt.scatter(upper_total_leaflet.positions[POPCi0:, 0],
            upper_total_leaflet.positions[POPCi0:, 1],
            marker = "x", color="k", alpha = 1, s=10)

#Plot coordinates of lipid assigned as ordered
#----------------------------------------------------------------------------------------------------------------------
plt.scatter(upper_total_leaflet.positions[ order_states_0 == 1, 0],
            upper_total_leaflet.positions[ order_states_0 == 1, 1],
            s = 35, marker = "o", facecolor="gray", zorder=-100, alpha=0.9)

#plt.scatter(upper_total_leaflet.positions[ order_states_0 == 0, 0],
#            upper_total_leaflet.positions[ order_states_0 == 0, 1],
#            s = 50, marker = "o", facecolor="gray", zorder=-100)


#Plot cosmetic
#----------------------------------------------------------------------------------------------------------------------
plt.ylim(-5, 248)
plt.xlim(-5, 248)

plt.xticks([])
plt.yticks([])

plt.scatter([], [], marker = "s", color = "red", label = "POPC")
plt.scatter([], [], marker = "^", color = "darkblue", label = "PUPC")
plt.scatter([], [], marker = "x", color = "black", label = "CHOL")

plt.legend(ncol=3, loc = "lower center", bbox_to_anchor=(0.5, -0.15), fontsize=15, frameon=False)
iax.axes[0].set_aspect("equal")

plt.title("f", fontsize=20, fontweight="bold", loc = "left")
plt.savefig("ORD_B_CHOL.pdf", transparent=True, dpi=400)

In [ ]:
#Init figure
iax = plt.figure()

#Select the last frame of the trajectory
i = -1
uni.trajectory[i]

#Leaflet Assignment
#----------------------------------------------------------------------------------------------------------------------
com_membrane = membrane.center_of_mass()

upper_leaflet = uni.select_atoms(f"name PO4 and prop z > {com_membrane[2]}")
lower_leaflet = uni.select_atoms(f"name PO4 and prop z <= {com_membrane[2]}")

upper_cholesterol = distances.distance_array(reference = cholesterol, configuration = upper_leaflet, box = ts.dimensions)
lower_cholesterol = distances.distance_array(reference = cholesterol, configuration = lower_leaflet, box = ts.dimensions)

upper_cholesterol = np.min(upper_cholesterol, axis = 1)
lower_cholesterol = np.min(lower_cholesterol, axis = 1)

upper_cholesterol = cholesterol[ upper_cholesterol < lower_cholesterol ]
lower_cholesterol = cholesterol.difference( upper_cholesterol )

upper_total_leaflet = upper_leaflet + upper_cholesterol
lower_total_leaflet = lower_leaflet + lower_cholesterol

#Get predicted states for each lipid
#----------------------------------------------------------------------------------------------------------------------
do_popc = predictPOPC[:, i][ leaflet_assignment[:, i][ idxPOPC ] == 0 ]
do_pupc = predictPUPC[:, i][ leaflet_assignment[:, i][ idxPUPC ] == 0 ]
do_chol = predictCHOL[:, i][ leaflet_assignment[:, i][ idxCHOL ] == 0 ]

order_states_0 = np.concatenate([do_pupc, do_popc, do_chol])

#Clustering
#----------------------------------------------------------------------------------------------------------------------
core_lipids = assign_core_lipids(weight_matrix_f = upper_weight_matrix[i],
                                 g_star_i_f      = g_star_i_0[i],
                                 order_states_f  = order_states_0,
                                 w_ii_f          = w_ii_0[i])

clusters = hierarchical_clustering(weight_matrix_f = upper_weight_matrix[i],
                                   w_ii_f          = w_ii_0[i], 
                                   core_lipids     = core_lipids)

#Plot coordinates
#----------------------------------------------------------------------------------------------------------------------
PUPCi0 = np.where(leaflet_assignment[ idxPUPC ][:, step]==0)[0].shape[0]
POPCi0 = np.where(leaflet_assignment[ idxPOPC ][:, step]==0)[0].shape[0] + PUPCi0

#Plot PUPC
plt.scatter(upper_total_leaflet.positions[:PUPCi0, 0],
            upper_total_leaflet.positions[:PUPCi0, 1],
            marker = "^", color="darkblue", alpha = 1, s=5)

#Plot POPC
plt.scatter(upper_total_leaflet.positions[PUPCi0:POPCi0, 0],
            upper_total_leaflet.positions[PUPCi0:POPCi0, 1],
            marker = "s", color="red", alpha = 1, s=5)

#Plot Cholesterol
plt.scatter(upper_total_leaflet.positions[POPCi0:, 0],
            upper_total_leaflet.positions[POPCi0:, 1],
            marker = "x", color="k", alpha = 1, s=15)

#----------------------------------------------------------------------------------------------------------------------

#First bin the coordinates
c, *_ = np.histogram2d(upper_total_leaflet.positions[:, 0],
                       upper_total_leaflet.positions[:, 1],
                       bins=12)

#Get for every coordinate the sum of the weights (aka G*i values)
s, xbin, ybin = np.histogram2d(upper_total_leaflet.positions[:, 0], upper_total_leaflet.positions[:, 1], bins=12, weights=g_star_i_0[i])

lims = [-5, 248, -5, 248]

iax = plt.imshow((s/c).T, extent=lims, origin='lower', cmap = "magma_r", alpha=1, interpolation="spline36", vmin=-1.5, vmax = 1.5)
iax.axes.set_aspect("equal")
cb = plt.colorbar(ax=iax.axes)
cb.set_label(r"$G^*_i$", fontsize = 18)
cb.ax.tick_params(labelsize=11) 

plt.ylim(-5, 248)
plt.xlim(-5, 248)


plt.xticks([])
plt.yticks([])

plt.scatter([], [], marker = "s", color = "red", label = "POPC")
plt.scatter([], [], marker = "^", color = "darkblue", label = "PUPC")
plt.scatter([], [], marker = "x", color = "black", label = "CHOL")

plt.legend(ncol=3, loc = "lower center", bbox_to_anchor=(0.5, -0.15), fontsize=15, frameon=False)

plt.title("d", fontsize=20, fontweight="bold", loc = "left")
plt.savefig("GO_XY_B_CHOL.pdf", transparent=True, dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))

for k, i in enumerate([0, 1000, 2000]):
    
    #Select the last frame of the trajectory
    uni.trajectory[i]
    
    #Leaflet Assignment
    #----------------------------------------------------------------------------------------------------------------------
    com_membrane = membrane.center_of_mass()
    
    upper_leaflet = uni.select_atoms(f"name PO4 and prop z > {com_membrane[2]}")
    lower_leaflet = uni.select_atoms(f"name PO4 and prop z <= {com_membrane[2]}")
    
    upper_cholesterol = distances.distance_array(reference = cholesterol, configuration = upper_leaflet, box = ts.dimensions)
    lower_cholesterol = distances.distance_array(reference = cholesterol, configuration = lower_leaflet, box = ts.dimensions)
    
    upper_cholesterol = np.min(upper_cholesterol, axis = 1)
    lower_cholesterol = np.min(lower_cholesterol, axis = 1)
    
    upper_cholesterol = cholesterol[ upper_cholesterol < lower_cholesterol ]
    lower_cholesterol = cholesterol.difference( upper_cholesterol )
    
    upper_total_leaflet = upper_leaflet + upper_cholesterol
    lower_total_leaflet = lower_leaflet + lower_cholesterol
    
    #Get predicted states for each lipid
    #----------------------------------------------------------------------------------------------------------------------
    do_popc = predictPOPC[:, i][ leaflet_assignment[:, i][ idxPOPC ] == 0 ]
    do_pupc = predictPUPC[:, i][ leaflet_assignment[:, i][ idxPUPC ] == 0 ]
    do_chol = predictCHOL[:, i][ leaflet_assignment[:, i][ idxCHOL ] == 0 ]
    
    order_states_0 = np.concatenate([do_pupc, do_popc, do_chol])
    
    #Clustering
    #----------------------------------------------------------------------------------------------------------------------
    core_lipids = assign_core_lipids(weight_matrix_f = upper_weight_matrix[i],
                                     g_star_i_f      = g_star_i_0[i],
                                     order_states_f  = order_states_0,
                                     w_ii_f          = w_ii_0[i])
    
    clusters = hierarchical_clustering(weight_matrix_f = upper_weight_matrix[i],
                                       w_ii_f          = w_ii_0[i], 
                                       core_lipids     = core_lipids)
    
    #Plot coordinates
    #----------------------------------------------------------------------------------------------------------------------
    PUPCi0 = np.where(leaflet_assignment[ idxPUPC ][:, step]==0)[0].shape[0]
    POPCi0 = np.where(leaflet_assignment[ idxPOPC ][:, step]==0)[0].shape[0] + PUPCi0
    
    #Plot PUPC
    ax[k].scatter(upper_total_leaflet.positions[:PUPCi0, 0],
                upper_total_leaflet.positions[:PUPCi0, 1],
                marker = "^", color="darkblue", alpha = 1, s=5)
    
    #Plot POPC
    ax[k].scatter(upper_total_leaflet.positions[PUPCi0:POPCi0, 0],
                upper_total_leaflet.positions[PUPCi0:POPCi0, 1],
                marker = "s", color="red", alpha = 1, s=5)
    
    #Plot Cholesterol
    ax[k].scatter(upper_total_leaflet.positions[POPCi0:, 0],
                upper_total_leaflet.positions[POPCi0:, 1],
                marker = "x", color="k", alpha = 1, s=15)
    
    #Choose color scheme for clustering coloring
    colors = plt.cm.viridis_r(np.linspace(0,1.0, 31))

    #Iterate over clusters and plot the residues
    print(f"Number of clusters in frame {i}: {len(clusters.values())}")
    for j, val in enumerate(clusters.values()):

        idx=np.array(list(val),dtype=int)
        ax[k].scatter(upper_total_leaflet.positions[ idx, 0],
                      upper_total_leaflet.positions[ idx, 1],
                      s = 100, marker = "o", color = colors[j], zorder =-10)

    #Plot cosmetics
    ax[k].set_ylim(-5, 248)
    ax[k].set_xlim(-5, 248)

    ax[k].set_xticks([])
    ax[k].set_yticks([])
    
    ax[k].scatter([], [], marker = "s", color = "red", label = "POPC")
    ax[k].scatter([], [], marker = "^", color = "darkblue", label = "PUPC")
    ax[k].scatter([], [], marker = "x", color = "black", label = "CHOL")
    
    ax[1].legend(ncol=3, loc = "lower center", bbox_to_anchor=(0.5, -0.15), fontsize=15, frameon=False)
    ax[k].set_aspect("equal")

plt.subplots_adjust(wspace=-0.45)
ax[0].set_title("d", fontsize=20, fontweight="bold", loc = "left")
ax[1].set_title("e", fontsize=20, fontweight="bold", loc = "left")
ax[2].set_title("f", fontsize=20, fontweight="bold", loc = "left")

ax[0].text(s = r"$t=8\, \mu s$",x=126.5,y=257, fontsize=18, ha ="center", va="center")
ax[1].text(s = r"$t=9\, \mu s$",x=126.5,y=257, fontsize=18, ha ="center", va="center")
ax[2].text(s = r"$t=10\, \mu s$",x=126.5,y=257, fontsize=18, ha ="center", va="center")
plt.savefig("CLU_B_CHOL.pdf", transparent=True, dpi=300)

# $L_o$ Domain Analysis

Apply the hierarchical clustering in each leaflet at each time step.

In [ ]:
#Upper leaflet
all_clusters_0 = []

#Iterate over frames
for i in tqdm(range( len( g_star_i_0 ) )):

    #Get predictions for each lipid in the frame
    do_popc = predictPOPC[:, i][ leaflet_assignment[:, i][ idxPOPC ] == 0 ]
    do_pupc = predictPUPC[:, i][ leaflet_assignment[:, i][ idxPUPC ] == 0 ]
    do_chol = predictCHOL[:, i][ leaflet_assignment[:, i][ idxCHOL ] == 0 ]

    #Concatenate the predictions to one large array
    order_states_0 = np.concatenate([do_pupc, do_popc, do_chol])
    
     #Assign lipids that belong to the core members
    core_lipids = assign_core_lipids(weight_matrix_f = upper_weight_matrix[i],
                                     g_star_i_f = g_star_i_0[i],
                                     order_states_f = order_states_0,
                                     w_ii_f = w_ii_0[i])

    #Apply the clustering
    clusters = hierarchical_clustering(weight_matrix_f = upper_weight_matrix[i],
                                       w_ii_f = w_ii_0[i], 
                                       core_lipids=core_lipids)
    all_clusters_0.append(clusters)

In [ ]:
#Upper leaflet
all_clusters_1 = []

#Iterate over frames
for i in tqdm(range( len( g_star_i_1 ) )):

    #Get predictions for each lipid in the frame
    do_popc = predictPOPC[:, i][ leaflet_assignment[:, i][ idxPOPC ] == 1 ]
    do_pupc = predictPUPC[:, i][ leaflet_assignment[:, i][ idxPUPC ] == 1 ]
    do_chol = predictCHOL[:, i][ leaflet_assignment[:, i][ idxCHOL ] == 1 ]

    #Concatenate the predictions to one large array
    order_states_1 = np.concatenate([do_pupc, do_popc, do_chol])
    
     #Assign lipids that belong to the core members
    core_lipids = assign_core_lipids(weight_matrix_f = lower_weight_matrix[i],
                                     g_star_i_f = g_star_i_1[i],
                                     order_states_f = order_states_1,
                                     w_ii_f = w_ii_1[i])

    #Apply the clustering
    clusters = hierarchical_clustering(weight_matrix_f = lower_weight_matrix[i],
                                       w_ii_f = w_ii_1[i], 
                                       core_lipids=core_lipids)
    all_clusters_1.append(clusters)

The last part of this notebook includes the analysis of the number, the size, and the lipid composition of the identified $L_o$ domains. The next coding cell again iterates over the trajectory and counts the number of each lipid type in the clusters. The values are then either compared to the total number of this lipid type in the leaflet ($\frac{\#L_o}{\#L_{Total}}$) or to the number of all lipids in all $L_o$ domains.

In [ ]:
#Initialize arrays to store the data
avgerage_number_chol = np.zeros( (n_frames, 2) )
avgerage_number_popc = np.zeros( (n_frames, 2) )
avgerage_number_pupc = np.zeros( (n_frames, 2) )

avgerage_comp_chol = np.zeros( (n_frames, 2) )
avgerage_comp_popc = np.zeros( (n_frames, 2) )
avgerage_comp_pupc = np.zeros( (n_frames, 2) )



for step, ts in tqdm( enumerate( uni.trajectory  ), total = n_frames ):

    if not step % 1:

        #Assign leaflets based on the center of mass of the membrane
        com_membrane = membrane.center_of_mass()

        #Select phospholipids in the upper and lower leaflet
        upper_leaflet = uni.select_atoms(f"name PO4 and prop z > {com_membrane[2]}")
        lower_leaflet = uni.select_atoms(f"name PO4 and prop z <= {com_membrane[2]}")

    #Assign cholesterol to a leaflet
    upper_cholesterol = distances.distance_array(reference = cholesterol, configuration = upper_leaflet, box = ts.dimensions)
    lower_cholesterol = distances.distance_array(reference = cholesterol, configuration = lower_leaflet, box = ts.dimensions)

    upper_cholesterol = np.min(upper_cholesterol, axis = 1)
    lower_cholesterol = np.min(lower_cholesterol, axis = 1)

    upper_cholesterol = cholesterol[ upper_cholesterol < lower_cholesterol ]
    lower_cholesterol = cholesterol.difference( upper_cholesterol )

    upper_total_leaflet = upper_leaflet + upper_cholesterol
    lower_total_leaflet = lower_leaflet + lower_cholesterol

    box = ts.dimensions

    #Get dictionaries containing the clusters of the current time step for the...
    upper_clusters = all_clusters_0[step]#...upper leaflet, and...
    lower_clusters = all_clusters_1[step]#...the lower leaflet.

    #Upper leaflet
    #----------------------------------------------------------------------------------------------
    #Number of lipid types in clusters
    npopc = 0
    npupc = 0
    nchol = 0

    #Iterate over clusters
    for cluster in upper_clusters.values():

        npopc += np.sum(upper_total_leaflet.resnames[cluster] == "POPC")
        npupc += np.sum(upper_total_leaflet.resnames[cluster] == "PUPC")
        nchol += np.sum(upper_total_leaflet.resnames[cluster] == "CHOL")
        
    #Get the ratio between number lipids in Lo and total number of lipids in leaflet
    avgerage_number_popc[step, 0] = npopc / np.sum(upper_total_leaflet.resnames == "POPC")
    avgerage_number_pupc[step, 0] = npupc / np.sum(upper_total_leaflet.resnames == "PUPC")
    avgerage_number_chol[step, 0] = nchol / np.sum(upper_total_leaflet.resnames == "CHOL")

    #Get the composition of the Lo domain
    avgerage_comp_popc[step, 0] = npopc / (npopc + npupc + nchol)
    avgerage_comp_pupc[step, 0] = npupc / (npopc + npupc + nchol)
    avgerage_comp_chol[step, 0] = nchol / (npopc + npupc + nchol)

    #Lower leaflet
    #----------------------------------------------------------------------------------------------
    #Number of lipid types in clusters
    npopc = 0
    npupc = 0
    nchol = 0

    #Iterate over clusters
    for cluster in lower_clusters.values():

        npopc += np.sum(lower_total_leaflet.resnames[cluster] == "POPC")
        npupc += np.sum(lower_total_leaflet.resnames[cluster] == "PUPC")
        nchol += np.sum(lower_total_leaflet.resnames[cluster] == "CHOL")
        

    #Get the ratio between number lipids in Lo and total number of lipids in leaflet
    avgerage_number_popc[step, 1] = npopc / np.sum(lower_total_leaflet.resnames == "POPC")
    avgerage_number_pupc[step, 1] = npupc / np.sum(lower_total_leaflet.resnames == "PUPC")
    avgerage_number_chol[step, 1] = nchol / np.sum(lower_total_leaflet.resnames == "CHOL")

    #Get the composition of the Lo domain
    avgerage_comp_popc[step, 1] = npopc / (npopc + npupc + nchol)
    avgerage_comp_pupc[step, 1] = npupc / (npopc + npupc + nchol)
    avgerage_comp_chol[step, 1] = nchol / (npopc + npupc + nchol)
    

In [ ]:
fig, ax = plt.subplots(1, 4, figsize = (25, 5))

t = np.linspace(8, 10, 2001)

number_of_clusters = []
membs_per_cluster = []

for clusters_0, clusters_1 in zip(all_clusters_0, all_clusters_1):

    no_clusters_0 = len(clusters_0.keys())
    no_clusters_1 = len(clusters_1.keys())
    
    number_of_clusters.append( (no_clusters_0  + no_clusters_1) / 2 )

    membs = []
    for cluster in clusters_0.values(): membs.append( len(cluster) )
    for cluster in clusters_1.values(): membs.append( len(cluster) )

    membs_per_cluster += membs
    

ax[0].plot(t, number_of_clusters, color= "gray")
ax[0].set_xticks([8., 8.5, 9., 9.5, 10.])
ax[0].set_xlim(8, 10)
ax[0].set_xlabel(r"$t~(\mu s)$", fontsize=18)
y0 = ax[0].set_ylabel(r"$\#\,Clusters$", fontsize=18)
ax[0].set_ylim(0, 25)
ax[0].set_yticks(np.linspace(0, 25, 6))

ax[1].hist(membs_per_cluster, density=True, bins = np.linspace(0, 270, 51), edgecolor="k", facecolor="red", alpha =0.5 )
ax[1].set_xlim(0, 270)
ax[1].set_ylim(0, .06)
ax[1].set_yticks(np.linspace(0, 0.06, 4))
ax[1].set_xlabel(r"$\#\,Members$", fontsize=18)
ax[1].set_ylabel(r"$p(\#\,Members)$", fontsize=18)

ax[2].hist(avgerage_number_chol.mean(1), density=True, bins = np.linspace(0, 1, 101), color="k", alpha =1, histtype="step", lw = 2)
ax[2].hist(avgerage_number_pupc.mean(1), density=True, bins = np.linspace(0, 1, 101), color="darkblue", alpha =1, histtype="step", lw = 2)
ax[2].hist(avgerage_number_popc.mean(1), density=True, bins = np.linspace(0, 1, 101), color="r", alpha =1, histtype="step", lw = 2)
ax[2].set_xlim(0, 1)

x2 = ax[2].set_xlabel(r'$\frac{\#\,L_o}{\#\,Total}$', fontsize=18)
ax[2].set_ylabel(r"$p \left ( \frac{\#\,L_o}{\#\,Total} \right ) $", fontsize=18)

ax[2].plot([], color = "r", lw=2, label = "POPC")
ax[2].plot([], color = "darkblue", lw=2, label = "PUPC")
ax[2].plot([], color = "k", lw=2, label = "Chol")
ax[2].legend(fontsize=15, loc = "upper right")
ax[2].set_yticks(np.linspace(0, 15, 4))
ax[2].set_ylim(0, 15)

ax[3].hist(avgerage_comp_chol.mean(1), density=True, bins = np.linspace(0, 1, 101), color="k", alpha =1, histtype="step", lw = 2)
ax[3].hist(avgerage_comp_pupc.mean(1), density=True, bins = np.linspace(0, 1, 101), color="darkblue", alpha =1, histtype="step", lw = 2)
ax[3].hist(avgerage_comp_popc.mean(1), density=True, bins = np.linspace(0, 1, 101), color="r", alpha =1, histtype="step", lw = 2)
ax[3].set_xlim(0, 1)
ax[3].set_xlabel(r"$Composition$", fontsize=18)
ax[3].set_ylabel(r"$p(Composition)$", fontsize=18)
ax[3].set_ylim(0, 40)
ax[3].set_yticks([0,10,20,30,40])

ax[3].plot([], color = "r", lw=2, label = "POPC")
ax[3].plot([], color = "darkblue", lw=2, label = "PUPC")
ax[3].plot([], color = "k", lw=2, label = "Chol")
ax[3].legend(fontsize=15, loc = "upper right")

ax[0].set_title("e", loc="left", fontsize=20, fontweight="bold")
ax[1].set_title("f", loc="left", fontsize=20, fontweight="bold")
ax[2].set_title("g", loc="left", fontsize=20, fontweight="bold")
ax[3].set_title("h", loc="left", fontsize=20, fontweight="bold")
plt.subplots_adjust(wspace=0.25)

for i in range(4):
    ax[i].tick_params(axis="both", labelsize=18)

plt.savefig("RES_B_CHOL.pdf", transparent=True, dpi=300, bbox_extra_artists=[x2,y0], bbox_inches="tight")